In [52]:
def type_of_rating(data):
    facilities = []
    for row in data[1:]:
        facility1 = row[0]
        facility = str(facility1).lower()
        print(facility)
        if 'short term' in facility or 'long term / short term' in facility:
            facilities.append('ST')
        elif 'long term' in facility or 'long-term'in facility or 'long - term'in facility:
            facilities.append('LT')
        elif 'unallocated term' in facility:
            facilities.append('UT')
        else:
            facilities.append('ST')
    return facilities

import re
import numpy as np

def extract_ratings_from_data(data):
    rating_pattern = re.compile(r'\b(?:[A-Z]{1,3}[+-]?/[A-Z0-9+-]+(?:/[A-Z0-9+-]+)?(?:;Negative)?)\b|[A-Z0-9+-]+(?:;Negative)?')
    ratings = []
    for row in data:
        row_ratings = []
        for cell in row:
            if isinstance(cell, str): 
                matches = rating_pattern.findall(cell)
                valid_matches = [match for match in matches if len(match) > 1 and not match.isdigit() and match not in ['CARE', 'ICRA', 'Commercial paper (CP)','CP','CE']]
                row_ratings.extend(valid_matches)
        if row_ratings:
            ratings.append('/'.join(row_ratings))
    
    return ratings

def extract_type_of_facilities_care(data):
    facilities = []
    pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|bank facilities|Term Loan|term loan|Commercial paper|commercial paper|Amount)[@#]?$')
    for row in data:
        if isinstance(row[0], str) and pattern.match(row[0]):
            facilities.append(row[0])
    
    return facilities

In [53]:

def extract_all_amount(data):
    short_term_amounts = []
    long_term_amounts = []
    unallocated_amounts = []
    
    for row in data:
        if isinstance(row[1], str) and row[1].replace('.', '', 1).isdigit():
            amount = float(row[1])
            facility1 = row[0]
            facility = facility1.lower()
            
            if 'short term' in facility or 'long term / short term' in facility or 'short-term' in facility:
                short_term_amounts.append(amount)
                long_term_amounts.append(0.00)
                unallocated_amounts.append(0.00)
            elif 'long term' in facility or 'long-term' in facility:
                long_term_amounts.append(amount)
                short_term_amounts.append(0.00)
                unallocated_amounts.append(0.00)
            else:
                unallocated_amounts.append(amount)
                short_term_amounts.append(0.00)
                long_term_amounts.append(0.00)
    
    return short_term_amounts, long_term_amounts, unallocated_amounts


def company_name(companies):
    com_name = []
    if 'CARE' in companies[0] or 'ICRA' in companies[0]:
        companies.pop(0)
    new_first_company = companies[0]
    new_first_company = re.sub(r'Press Release\s+', '', new_first_company, flags=re.IGNORECASE)
    new_first_company = re.sub(r'^\d+\s+', '', new_first_company)
    pattern = re.compile(r'\b[A-Za-z\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
    matches = pattern.findall(new_first_company)
    expected_company_name = matches[0] if matches else ""
    expected_company_name = expected_company_name.replace('\n', ' ')
    com_name.append(expected_company_name)
    return com_name


def outlook(data):
    ratings = []
    for row in data[1:]:
        rating = row[2]
        if 'Stable' in rating:
            ratings.append('Stable')
        elif 'Positive' in rating or 'Negative' in rating:
            ratings.append(rating.split(';')[1].strip())
        else:
            ratings.append('-')

    return ratings


def extract_text_from_last_pages(pdf_file_path, num_pages=3):
    reader = PdfReader(pdf_file_path)
    text = ""
    total_pages = len(reader.pages)
    for page_num in range(total_pages - num_pages, total_pages):
        text += reader.pages[page_num].extract_text()
    return text

def extract_analyst_contacts(pdf_file_path):
    text = extract_text_from_last_pages(pdf_file_path)
    name_pattern = re.compile(r'(?<=\n)[A-Za-z\s]+(?=\n\+\d{2}\s\d{2,3}\s\d{7,10})')
    email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
    phone_pattern = re.compile(r'\+\d{2}\s\d{2,3}\s\d{7,10}')
    names = name_pattern.findall(text)
    emails = email_pattern.findall(text)
    phones = phone_pattern.findall(text)
    names_list = []
    emails_list = []
    contact_numbers_list = []
    for name, phone in zip(names, phones):
        email_match = email_pattern.search(text, text.find(phone))
        email = email_match.group(0) if email_match else None
        names_list.append(name.strip())
        emails_list.append(email)
        contact_numbers_list.append(phone)

    return names_list, emails_list, contact_numbers_list


from PyPDF2 import PdfReader
import numpy as np
def remove_commas(data):
    for row in data:
        for i in range(len(row)):
            if isinstance(row[i], str) and row[i].replace(',', '').replace('.', '').isdigit():
                row[i] = row[i].replace(',', '')
    return data

from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_file_path):
    reader = PdfReader(pdf_file_path)
    page = reader.pages[0]
    text = page.extract_text()
    return text

def find_agency_names(pdf_file_path):
    text = extract_text_from_pdf(pdf_file_path)
    agencies = []
    if 'CARE' in text:
        agencies.append('CARE')
    if 'ICRA' in text:
        agencies.append('ICRA')
    if 'Brickwork rating' in text or 'brickworkrating' in text:
        agencies.append('Brickworks')
    return agencies

In [54]:
def extract_text_from_last_pages(pdf_file_path, num_pages=3):
    reader = PdfReader(pdf_file_path)
    text = ""
    total_pages = len(reader.pages)
    for page_num in range(total_pages - num_pages, total_pages):
        text += reader.pages[page_num].extract_text()
    return text

def extract_analyst_contacts(pdf_file_path):
    text = extract_text_from_last_pages(pdf_file_path)
    
    if "ANALYST CONTACT" in text:
        name_pattern = re.compile(r'(?<=\n)[A-Za-z\s]+(?=\n\+\d{2}\s\d{2,3}\s\d{7,10})')
        email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
        phone_pattern = re.compile(r'\+\d{2}\s\d{2,3}\s\d{7,10}')
        names = name_pattern.findall(text)
        emails = email_pattern.findall(text)
        phones = phone_pattern.findall(text)
        names_list = []
        emails_list = []
        contact_numbers_list = []
        
        for name, phone in zip(names, phones):
            email_match = email_pattern.search(text, text.find(phone))
            email = email_match.group(0) if email_match else None
            names_list.append(name.strip())
            emails_list.append(email)
            contact_numbers_list.append(phone)

        return names_list, emails_list, contact_numbers_list
    else:
        return ['-'], ['-'], ['-']


In [61]:

xl_path = r"D:\Python\ajit\project_credit_rating\try\auto\excel\Adani Agri Fresh Limited 1.xlsx"
pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Agri Fresh Limited 1.pdf"
# pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\auto\care\pdf\Adani Agri Logistics Limited 1.pdf"
# xl_path = r"D:\Python\ajit\project_credit_rating\try\auto\care\excel\Adani Agri Logistics Limited 1.xlsx"

com1 = []
age=[]
name = []
email = []
contact_number = []
ty_faci = []
xls = pd.ExcelFile(xl_path)
table_name = xls.sheet_names
df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
a1 = df1.values.tolist()
data = remove_commas(a1)
outlook1 = outlook(a1)
short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(data)
ratings = extract_ratings_from_data(a1)
rat1 = []
# if ratings != rating_type:
#     rat1.extend(ratings * len(rating_type))
# else:
#     rat1 = ratings
rating_type = type_of_rating(a1)
type_facility=extract_type_of_facilities_care(a1)
ty_faci.extend(type_facility*len(rating_type))
print(type_facility)
names, emails, contact_numbers = extract_analyst_contacts(pdf_file_path)
name.extend(names*len(rating_type))
email.extend(emails*len(rating_type))
contact_number.extend(contact_numbers*len(rating_type))
agencies = find_agency_names(pdf_file_path)
text = extract_text_from_pdf(pdf_file_path)
pattern = re.compile(r'\b[A-Za-z0-9\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
companies = pattern.findall(text)
com = company_name(companies)
com1.extend(com * len(rating_type))
age.extend(agencies * len(rating_type))

print(len(com1),len(age),len(type_facility),len(rating_type),len(outlook1),len(ratings),len(short_term_amounts),
      len(long_term_amounts),len(unallocated_amounts),len(name),len(email),len(contact_number))

data = {
'Company_Name': com1,
'Rating_Agency': age,
'Type_of_facilities': type_facility,
'type_of_rating': rating_type,
'Outlook' : outlook1,
'Rating':ratings,
'short_term_amounts':short_term_amounts,
'long_term_amounts':long_term_amounts,
'unallocated_amounts':unallocated_amounts,
'Analyst_name': name,
'Email': email,
'Contact_Number' :contact_number
    }

# Create the DataFrame
df = pd.DataFrame(data)
df

long term / short term bank facilities
short term bank facilities
['Long Term / Short Term Bank Facilities', 'Short Term Bank Facilities']
2 2 2 2 2 2 2 2 2 2 2 2


,Company_Name,Rating_Agency,Type_of_facilities,type_of_rating,Outlook,Rating,short_term_amounts,long_term_amounts,unallocated_amounts,Analyst_name,Email,Contact_Number
0,Adani Agri Fresh Limited,CARE,Long Term / Short Term Bank Facilities,ST,Stable,BBB-/A3,145.0,0.0,0.0,-,-,-
1,Adani Agri Fresh Limited,CARE,Short Term Bank Facilities,ST,-,A3,10.0,0.0,0.0,-,-,-


In [67]:
import os
import pandas as pd
import re

# Directory paths
pdf_directory = r"D:\Python\ajit\project_credit_rating\try\auto\care\pdf"
excel_directory = r"D:\Python\ajit\project_credit_rating\try\auto\care\excel"
op_csv = r'D:\Python\ajit\project_credit_rating\try\auto\care\op_csv'

# Iterate over each Excel file in the directory
for excel_file in os.listdir(excel_directory):
    if excel_file.endswith(".xlsx"):
        # Initialize lists to store data for each file
        xl_path = os.path.join(excel_directory, excel_file)
        pdf_file_path = os.path.join(pdf_directory, excel_file.replace(".xlsx", ".pdf"))
        com1 = []
        age = []
        name = []
        email = []
        contact_number = []
        ty_faci = []
        xls = pd.ExcelFile(xl_path)
        table_name = xls.sheet_names
        df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
        a1 = df1.values.tolist()
        data = remove_commas(a1)
        outlook1 = outlook(a1)
        short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(data)
        ratings = extract_ratings_from_data(a1)
        rat1 = []
        # if ratings != rating_type:
        #     rat1.extend(ratings * len(rating_type))
        # else:
        #     rat1 = ratings
        rating_type = type_of_rating(a1)
        type_facility = extract_type_of_facilities_care(a1)
        ty_faci.extend(type_facility * len(rating_type))
        print(type_facility)
        names, emails, contact_numbers = extract_analyst_contacts(pdf_file_path)
        name.extend(names * len(rating_type))
        email.extend(emails * len(rating_type))
        contact_number.extend(contact_numbers * len(rating_type))
        agencies = find_agency_names(pdf_file_path)
        text = extract_text_from_pdf(pdf_file_path)
        pattern = re.compile(r'\b[A-Za-z0-9\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
        companies = pattern.findall(text)
        com = company_name(companies)
        com1.extend(com * len(rating_type))
        age.extend(agencies * len(rating_type))

        print(len(com1), len(age), len(type_facility), len(rating_type), len(outlook1), len(ratings), len(short_term_amounts),
              len(long_term_amounts), len(unallocated_amounts), len(name), len(email), len(contact_number))

        data = {
            'Company_Name': com1,
            'Rating_Agency': age,
            'Type_of_facilities': type_facility,
            'type_of_rating': rating_type,
            'Outlook': outlook1,
            'Rating': ratings,
            'short_term_amounts': short_term_amounts,
            'long_term_amounts': long_term_amounts,
            'unallocated_amounts': unallocated_amounts,
            'Analyst_name': name,
            'Email': email,
            'Contact_Number': contact_number
        }

        # Create the DataFrame
        df = pd.DataFrame(data)
        print(df)
        print('\n=====================')

        # Save the DataFrame to a CSV file in the specified folder
        output_csv_path = os.path.join(op_csv, excel_file.replace(".xlsx", ".csv"))
        df.to_csv(output_csv_path, index=False)

print("Data has been successfully saved to separate CSV files for each PDF.")


long term / short term bank facilities
short term bank facilities
['Long Term / Short Term Bank Facilities', 'Short Term Bank Facilities']
2 2 2 2 2 2 2 2 2 2 2 2
               Company_Name Rating_Agency  \
0  Adani Agri Fresh Limited          CARE   
1  Adani Agri Fresh Limited          CARE   

                       Type_of_facilities type_of_rating Outlook   Rating  \
0  Long Term / Short Term Bank Facilities             ST  Stable  BBB-/A3   
1              Short Term Bank Facilities             ST       -       A3   

   short_term_amounts  long_term_amounts  unallocated_amounts Analyst_name  \
0               145.0                0.0                  0.0            -   
1                10.0                0.0                  0.0            -   

  Email Contact_Number  
0     -              -  
1     -              -  

long-term bank facilities@
['Long-term bank facilities@']
1 1 1 1 1 1 1 1 1 1 1 1
                   Company_Name Rating_Agency          Type_of_facilities  \

In [ ]:
import os
import pandas as pd
import re


# Directory paths
pdf_directory = r"D:\Python\ajit\project_credit_rating\try\auto\care\pdf"
excel_directory = r"D:\Python\ajit\project_credit_rating\try\auto\care\excel"

# Initialize lists to store data
com1 = []
age = []
name = []
email = []
contact_number = []
ty_faci = []
rating_type_list = []
outlook_list = []
ratings_list = []
short_term_amounts_list = []
long_term_amounts_list = []
unallocated_amounts_list = []

# Iterate over each Excel file in the directory
for excel_file in os.listdir(excel_directory):
    if excel_file.endswith(".xlsx"):
        xl_path = os.path.join(excel_directory, excel_file)
        pdf_file_path = os.path.join(pdf_directory, excel_file.replace(".xlsx", ".pdf"))

        xls = pd.ExcelFile(xl_path)
        df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
        a1 = df1.values.tolist()
        data = remove_commas(a1)
        outlook1 = outlook(a1)
        short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(data)
        ratings = extract_ratings_from_data(a1)
        rating_type = type_of_rating(a1)
        type_facility = extract_type_of_facilities_care(a1)
        
        ty_faci.extend(type_facility * len(rating_type))
        rating_type_list.extend(rating_type)
        outlook_list.extend(outlook1)
        ratings_list.extend(ratings)
        short_term_amounts_list.extend(short_term_amounts)
        long_term_amounts_list.extend(long_term_amounts)
        unallocated_amounts_list.extend(unallocated_amounts)
        
        names, emails, contact_numbers = extract_analyst_contacts(pdf_file_path)
        name.extend(names * len(rating_type))
        email.extend(emails * len(rating_type))
        contact_number.extend(contact_numbers * len(rating_type))
        
        agencies = find_agency_names(pdf_file_path)
        text = extract_text_from_pdf(pdf_file_path)
        pattern = re.compile(r'\b[A-Za-z0-9\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
        companies = pattern.findall(text)
        com = company_name(companies)
        com1.extend(com * len(rating_type))
        age.extend(agencies * len(rating_type))

# Ensure all lists are of the same length
max_length = max(len(com1), len(age), len(ty_faci), len(rating_type_list), len(outlook_list), len(ratings_list), len(short_term_amounts_list), len(long_term_amounts_list), len(unallocated_amounts_list), len(name), len(email), len(contact_number))

com1.extend([""] * (max_length - len(com1)))
age.extend([""] * (max_length - len(age)))
ty_faci.extend([""] * (max_length - len(ty_faci)))
rating_type_list.extend([""] * (max_length - len(rating_type_list)))
outlook_list.extend([""] * (max_length - len(outlook_list)))
ratings_list.extend([""] * (max_length - len(ratings_list)))
short_term_amounts_list.extend([0] * (max_length - len(short_term_amounts_list)))
long_term_amounts_list.extend([0] * (max_length - len(long_term_amounts_list)))
unallocated_amounts_list.extend([0] * (max_length - len(unallocated_amounts_list)))
name.extend([""] * (max_length - len(name)))
email.extend([""] * (max_length - len(email)))
contact_number.extend([""] * (max_length - len(contact_number)))

# Create the DataFrame with the collected data
data = {
    'Company_Name': com1,
    'Rating_Agency': age,
    'Type_of_facilities': ty_faci,
    'type_of_rating': rating_type_list,
    'Outlook': outlook_list,
    'Rating': ratings_list,
    'short_term_amounts': short_term_amounts_list,
    'long_term_amounts': long_term_amounts_list,
    'unallocated_amounts': unallocated_amounts_list,
    'Analyst_name': name,
    'Email': email,
    'Contact_Number': contact_number
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_csv_path = r"D:\Python\ajit\project_credit_rating\try\auto\care\output.csv"
# df.to_csv(output_csv_path, index=False)

print(f"Data has been successfully saved to {output_csv_path}")
